In [1]:
!pip install git+https://github.com/ekkus93/bark.git

  Cloning https://github.com/ekkus93/bark.git to /tmp/pip-req-build-k3xel1yz
  Running command git clone --filter=blob:none --quiet https://github.com/ekkus93/bark.git /tmp/pip-req-build-k3xel1yz
  Resolved https://github.com/ekkus93/bark.git to commit 965cbc1ce928b7b779dc6716235d19aeb533f6e9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12

In [2]:
!pip install git+https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer

['Collecting git+https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer',
 '  Cloning https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer to /tmp/pip-req-build-q35b7806',
 '  Running command git clone --filter=blob:none --quiet https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer /tmp/pip-req-build-q35b7806',
 '  Resolved https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer to commit 4f42e44480fb076a52ddeb1f5ec6132d3c1ad25a',
 '  Preparing metadata (setup.py) ... \x1b[?25l\x1b[?25hdone',
 'Collecting audiolm-pytorch==1.1.4 (from bark-hubert-quantizer==0.0.4)',
 '  Downloading audiolm_pytorch-1.1.4-py3-none-any.whl (37 kB)',
 'Collecting fairseq (from bark-hubert-quantizer==0.0.4)',
 '  Downloading fairseq-0.12.2.tar.gz (9.6 MB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/9.6 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.3/9.6 MB\x1b \x1b8.1

In [1]:
from bark.generation import load_codec_model, generate_text_semantic
from encodec.utils import convert_audio

import torchaudio
import torch

device = 'cuda' # or 'cpu'
model = load_codec_model(use_gpu=True if device == 'cuda' else False)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
# From https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
from hubert.hubert_manager import HuBERTManager
hubert_manager = HuBERTManager()
hubert_manager.make_sure_hubert_installed()
hubert_manager.make_sure_tokenizer_installed()

'data/models/hubert/tokenizer.pth'

In [3]:
# From https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
# Load HuBERT for semantic tokens
from hubert.pre_kmeans_hubert import CustomHubert
from hubert.customtokenizer import CustomTokenizer

# Load the HuBERT model
hubert_model = CustomHubert(checkpoint_path='data/models/hubert/hubert.pt').to(device)

# Load the CustomTokenizer model
tokenizer = CustomTokenizer.load_from_checkpoint('data/models/hubert/tokenizer.pth').to(device)  # Automatically uses the right layers

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [4]:
# Load and pre-process the audio waveform
#audio_filepath = 'audio.wav' # the audio you want to clone (under 13 seconds)
audio_filepath = 'benefits.wav'
wav, sr = torchaudio.load(audio_filepath)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.to(device)

In [5]:
semantic_vectors = hubert_model.forward(wav, input_sample_hz=model.sample_rate)
semantic_tokens = tokenizer.get_token(semantic_vectors)

In [6]:
# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav.unsqueeze(0))
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()  # [n_q, T]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


In [7]:
# move codes to cpu
codes = codes.cpu().numpy()
# move semantic tokens to cpu
semantic_tokens = semantic_tokens.cpu().numpy()

In [8]:
import numpy as np
voice_name = 'en_minah_0' # whatever you want the name of the voice to be
#output_path = 'bark/assets/prompts/' + voice_name + '.npz'
output_path = '/usr/local/lib/python3.10/dist-packages/bark/assets/prompts/' + voice_name + '.npz'

np.savez(output_path, fine_prompt=codes, coarse_prompt=codes[:2, :], semantic_prompt=semantic_tokens)

In [ ]:
# That's it! Now you can head over to the generate.ipynb and use your voice_name for the 'history_prompt'

In [ ]:
# Heres the generation stuff copy-pasted for convenience

In [13]:
from bark.api import generate_audio
from transformers import BertTokenizer
from bark.generation import SAMPLE_RATE, preload_models, codec_decode, generate_coarse, generate_fine, generate_text_semantic

# Enter your prompt and speaker here
text_prompt = "Hello, my name is Minah. And, uh — and I like pizza. [laughs]"
voice_name = "en_minah_0" # use your custom voice name here if you have one

In [14]:
# download and load all models
preload_models(
    text_use_gpu=True,
    text_use_small=False,
    coarse_use_gpu=True,
    coarse_use_small=False,
    fine_use_gpu=True,
    fine_use_small=False,
    codec_use_gpu=True,
    force_reload=False,
    #path="models"
)

In [15]:
import torch

# Check PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)

if cuda_available:
    # Check CUDA version
    print("CUDA version:", torch.version.cuda)
    # Check GPU details
    print("GPU details:", torch.cuda.get_device_name(0))

# Check cuDNN version
print("cuDNN version:", torch.backends.cudnn.version())


PyTorch version: 2.3.0+cu121
CUDA available: True
CUDA version: 12.1
GPU details: Tesla T4
cuDNN version: 8902


In [16]:
!cat /etc/os-release


PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [ ]:
#!sudo cp /var/cudnn-local-repo-ubuntu2204-8.9.7.29/cudnn-local-08A7D361-keyring.gpg /usr/share/keyrings/

In [ ]:
#!sudo dpkg -i /content/cudnn-local-repo-ubuntu2204-8.9.7.29_1.0-1_amd64.deb

(Reading database ... 123602 files and directories currently installed.)
Preparing to unpack .../cudnn-local-repo-ubuntu2204-8.9.7.29_1.0-1_amd64.deb ...
Unpacking cudnn-local-repo-ubuntu2204-8.9.7.29 (1.0-1) over (1.0-1) ...
Setting up cudnn-local-repo-ubuntu2204-8.9.7.29 (1.0-1) ...


In [20]:
# simple generation
audio_array = generate_audio(text_prompt, history_prompt=voice_name, text_temp=0.7, waveform_temp=0.7)

100%|██████████| 25/25 [00:27<00:00,  1.08s/it]


In [18]:
# generation with more control
x_semantic = generate_text_semantic(
    text_prompt,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)

x_coarse_gen = generate_coarse(
    x_semantic,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)
x_fine_gen = generate_fine(
    x_coarse_gen,
    history_prompt=voice_name,
    temp=0.5,
)
audio_array = codec_decode(x_fine_gen)

100%|██████████| 22/22 [06:07<00:00, 16.68s/it]


In [21]:
from IPython.display import Audio
# play audio
Audio(audio_array, rate=SAMPLE_RATE)

In [ ]:
from scipy.io.wavfile import write as write_wav
# save audio
#filepath = "/output/audio.wav" # change this to your desired output path
filepath = "output/audio.wav"
write_wav(filepath, SAMPLE_RATE, audio_array)